# Visualize GRIB file weather

In [1]:
import glob

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from mpl_toolkits.basemap import Basemap

In [2]:
files = glob.glob("weather/*.grb")
for file in files:
    ds = xr.open_dataset(file, engine="cfgrib")

lons = ds["longitude"]
lats = ds["latitude"]
wind_magnitude = np.sqrt(ds["u10"] ** 2 + ds["v10"] ** 2)
wind_direction = np.rad2deg(np.arctan(-ds["v10"] / (ds["u10"] + 0.001))) - 90.0

m = Basemap(
    llcrnrlon=min(lons),
    llcrnrlat=min(lats),
    urcrnrlon=max(lons),
    urcrnrlat=max(lats),
    resolution="h",
    projection="merc",
)

In [3]:
def plot_func(i):
    m.drawcoastlines()
    m.fillcontinents()
    alons, alats = np.meshgrid(lons, lats)

    # Velocity
    x, y = m(alons, alats)
    m.contourf(x, y, wind_magnitude[i], cmap="magma", alpha=0.5)

    # Plot wind barbs
    alons, alats = np.meshgrid(lons, lats)
    xx, yy = m(alons, alats)
    vx = -wind_magnitude[i] * np.sin(np.deg2rad(wind_direction[i]))
    vy = -wind_magnitude[i] * np.cos(np.deg2rad(wind_direction[i]))
    m.barbs(xx[::2, ::2], yy[::2, ::2], vx[::2, ::2], vy[::2, ::2], length=5, color="k")

    plt.colorbar()
    plt.show()


widgets.interact(
    plot_func, i=widgets.IntSlider(value=0, min=0, max=len(ds["u10"]) - 1, step=1)
)

interactive(children=(IntSlider(value=0, description='i', max=168), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_func(i)>